#### Incremental Learning- Model Retraining Approach

In [1]:
import pandas as pd
messages = pd.read_csv('smsspamcollection/SMSSpamCollection', sep='\t',
                           names=["label", "message"])

In [2]:
messages.shape

(5572, 2)

In [3]:
from sklearn.model_selection import train_test_split
message_train,message_test=train_test_split(messages)# both having label and message

In [4]:
message_train.head()

,label,message
5367,ham,Just trying to figure out when I'm suppose to ...
3698,spam,You are a winner you have been specially selec...
4050,ham,Yeah that's the impression I got
4435,ham,Sounds great! Im going to sleep now. Have a go...
320,ham,"Coffee cake, i guess..."


In [5]:
# Convert dataframe to list of tuples
messages_train = message_train.to_records(index=False)
messages_test=message_test.to_records(index=False)

In [6]:
messages_train[0]

('ham', "Just trying to figure out when I'm suppose to see a couple different people this week. We said we'd get together but I didn't set dates")

In [8]:
# Creating the pipeline
# 1st function is creating the TFIDF
# 2nd function is the naive bayes predictor
from creme import compose
from creme import feature_extraction
from creme import naive_bayes
from creme import metrics

model = compose.Pipeline(
    ('tokenize', feature_extraction.TFIDF(lowercase=False)),
    ('nb', naive_bayes.MultinomialNB(alpha=1))
)

In [11]:
metric=metrics.Accuracy()

# Training the model row by row
for label,sentence in messages_train:
    model = model.fit_one(sentence, label)
    y_pred = model.predict_one(sentence)
    metric = metric.update(label, y_pred)

### Training Data Accuracy
metric

Accuracy: 98.56%

In [13]:
### test Data Accuracy
test_metric=metrics.Accuracy()
for label,sentence in messages_test:
    y_pred = model.predict_one(sentence)
    test_metric = metric.update(label, y_pred)

### test Metric
test_metric

Accuracy: 98.16%

In [21]:
print(model.predict_one('This guy is neutral'))
print(model.predict_one('Everybody is neutral'))

In [17]:
model.fit_one("This guy is neutral", "ham")
model.fit_one("Everybody is neutral ", "ham")

Pipeline (
  TFIDF (
    normalize=True
    on=None
    strip_accents=True
    lowercase=False
    preprocessor=None
    tokenizer=<built-in method findall of re.Pattern object at 0x000002BF19093370>
    ngram_range=(1, 1)
  ),
  MultinomialNB (
    alpha=1
  )
)